### parseDiamond 

`prefix` `qseqid` `pident` `length`

`prefix` `kraken2-taxa`

In [1]:
import pandas as pd

In [10]:
demo = "/home/junyuchen/1-Projects/WGS-Pipeline/result/BGCs_demo/DA_100_blasp.tsv"

In [13]:
df = pd.read_table(demo, header=None)
df.columns = ["qseqid", "sseqid", "pident", "length", "mismatch", "gapopen", "qstart", "qend", "sstart", "send", "evalue", "bitscore"]

In [14]:
df

,qseqid,sseqid,pident,length,mismatch,gapopen,qstart,qend,sstart,send,evalue,bitscore
0,GKIHNHHK_03810,CAH08161.1,34.5,380,247,2,16,395,17,394,2.300000e-65,233.0
1,GKIHNHHK_04334,CAH08161.1,76.4,394,93,0,1,394,1,394,1.600000e-183,625.5


In [20]:
df1 = df.loc[df["pident"] >= 50]

In [21]:
df1

,qseqid,sseqid,pident,length,mismatch,gapopen,qstart,qend,sstart,send,evalue,bitscore
1,GKIHNHHK_04334,CAH08161.1,76.4,394,93,0,1,394,1,394,1.600000e-183,625.5


In [22]:
idList = df1["qseqid"].to_list()

In [23]:
idList

['GKIHNHHK_04334']

In [52]:
def parseDiamond(tsvFile):
    idList = []
    pidentList = []
    lengthList = []
    try:
        df = pd.read_table(tsvFile, header=None)
        df.columns = ["qseqid", "sseqid", "pident", "length", "mismatch", "gapopen", "qstart", "qend", "sstart", "send", "evalue", "bitscore"]
        df1 = df.loc[df["pident"] >= 50]
        idList = df1["qseqid"].to_list()
        pidentList = df1["pident"].to_list()
        lengthList = df1["length"].to_list()
    except Exception:
        pass
    return idList, pidentList, lengthList #bug

### extract seq

`seqid` and `prefix`

In [15]:
!ls /home/junyuchen/1-Projects/WGS-Pipeline/result/210202_WGS/prokka/DA_100 -alh

total 53M
drwxrwxr-x  2 junyuchen junyuchen 4.0K Feb  5 01:56 .
drwxrwxr-x 80 junyuchen junyuchen 4.0K Feb  5 04:01 ..
-rw-rw-r--  1 junyuchen junyuchen 1.1M Feb  5 01:54 DA_100.err
-rw-rw-r--  1 junyuchen junyuchen 1.7M Feb  5 01:54 DA_100.faa
-rw-rw-r--  1 junyuchen junyuchen 4.7M Feb  5 01:58 DA_100.ffn
-rw-rw-r--  1 junyuchen junyuchen 5.1M Feb  5 01:49 DA_100.fna
-rw-rw-r--  1 junyuchen junyuchen 5.1M Feb  5 01:54 DA_100.fsa
-rw-rw-r--  1 junyuchen junyuchen  11M Feb  5 02:00 DA_100.gbk
-rw-rw-r--  1 junyuchen junyuchen 6.5M Feb  5 01:54 DA_100.gff
-rw-rw-r--  1 junyuchen junyuchen  41K Feb  5 02:00 DA_100.log
-rw-rw-r--  1 junyuchen junyuchen  17M Feb  5 02:00 DA_100.sqn
-rw-rw-r--  1 junyuchen junyuchen 975K Feb  5 01:58 DA_100.tbl
-rw-rw-r--  1 junyuchen junyuchen 394K Feb  5 01:54 DA_100.tsv
-rw-rw-r--  1 junyuchen junyuchen  124 Feb  5 01:54 DA_100.txt


In [16]:
!grep "GKIHNHHK_04334" /home/junyuchen/1-Projects/WGS-Pipeline/result/210202_WGS/prokka/DA_100/DA_100.faa

>GKIHNHHK_04334 8-amino-7-oxononanoate synthase


In [17]:
from Bio import SeqIO

In [18]:
faaFile = "/home/junyuchen/1-Projects/WGS-Pipeline/result/210202_WGS/prokka/DA_100/DA_100.faa"

In [30]:
seqList = []
for seqid in idList:
    for seq in SeqIO.parse(faaFile, "fasta"):
        if seqid == seq.id:
            #print(seq)
            seqList.append(seq)

In [31]:
seqList

[SeqRecord(seq=Seq('MKLLQEKLSKYDAPQRAMAAGIYPYFRMIESDQDTEVMISGKKVLMFGSNAYLG...LVK', SingleLetterAlphabet()), id='GKIHNHHK_04334', name='GKIHNHHK_04334', description='GKIHNHHK_04334 8-amino-7-oxononanoate synthase', dbxrefs=[])]

In [26]:
OutDir = "/home/junyuchen/1-Projects/WGS-Pipeline/result/BGCs_demo_fasta"
prefix = "DA_100"

In [33]:
import os

In [34]:
SeqIO.write(seqList, os.path.join(OutDir, prefix + ".fasta"), "fasta")

1

In [54]:
def ExtractSeq(idList, prefix, faaFile, OutDir):
    seqList = []
    if len(idList) > 0:
        for seqid in idList:
            for seq in SeqIO.parse(faaFile, "fasta"):
                if seqid == seq.id:
                    #print(seq)
                    seqList.append(seq)
        SeqIO.write(seqList, os.path.join(OutDir, prefix + ".fasta"), "fasta")

In [55]:
InDir = "/home/junyuchen/1-Projects/WGS-Pipeline/result/BGCs_demo"

In [56]:
prokkaDir = "/home/junyuchen/1-Projects/WGS-Pipeline/result/210202_WGS/prokka"
OutDir = "/home/junyuchen/1-Projects/WGS-Pipeline/result/BGCs_demo_fasta_1"

In [60]:
df = pd.DataFrame()
for subdir, dirs, files in os.walk(InDir):
    for file in files:
        if file.endswith("_blasp.tsv"):
            filePath = os.path.join(subdir, file)
            prefix = file.replace("_blasp.tsv", "")
            #print(filePath)
            idList = []
            idList, pidentList, lengthList = parseDiamond(filePath)
            for i in range(len(idList)):
                df = df.append({"SampleID":prefix, "seqid":idList[i], "pident":str(pidentList[i]), "length":str(lengthList[i])}, ignore_index=True)
            faaFile = os.path.join(prokkaDir, prefix, prefix + ".faa")
            #ExtractSeq(idList, prefix, faaFile, OutDir)

In [61]:
df

,SampleID,length,pident,seqid
0,DA_258,407,60.7,DPHFJIJL_01467
1,DA_185,394,75.9,NHNCCHMJ_01069
2,DA_283,394,75.9,EBGFKMED_01067
3,DA_47,394,75.6,OAHAKLNK_01482
4,DA_566,394,75.6,MGJGFKBP_02929
...,...,...,...,...
57,DA_635,394,75.6,PJAALJFM_01318
58,DA_500,394,99.7,FBHHJDIK_01667
59,DA_524,394,74.6,FGPGDJNC_04468
60,DA_516,394,74.6,IFLMICDK_04493


In [63]:
df["SampleID"].describe()

count        62
unique       59
top       DA_61
freq          3
Name: SampleID, dtype: object

In [65]:
df.loc[df["SampleID"] == "DA_61"]

,SampleID,length,pident,seqid
21,DA_61,394,75.6,FNHMHEDM_06140
22,DA_61,382,75.7,FNHMHEDM_10324
23,DA_61,394,72.8,FNHMHEDM_12082
